In [5]:
import os

os.getpid() #- to get the current process running the jupyter notebook


1478

In [7]:
!sysctl hw.physicalcpu hw.logicalcpu

hw.physicalcpu: 4
hw.logicalcpu: 4


### Find the number of GPU devises available
Note that this does not recognize AMD GPU on iMac and macbooks

In [7]:
from catboost.utils import get_gpu_device_count
print('I see %i GPU devices' % get_gpu_device_count())

I see 0 GPU devices


In [6]:
import tensorflow as tf
print("GPU Available :",tf.test.is_gpu_available())
print("GPU devise name :",tf.test.gpu_device_name())
print("Built with CUDA :", tf.test.is_built_with_cuda())
print("Built with GPU Support :",tf.test.is_built_with_gpu_support())

GPU Available : False
GPU devise name : 
Built with CUDA : False
Built with GPU Support : False


### Check if OpenBLAS or MKL is included in the numpy install


In [12]:
import numpy as np
print(np.__version__)
np.show_config()

1.17.4
blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/opt/anaconda3/envs/regular/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/opt/anaconda3/envs/regular/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/opt/anaconda3/envs/regular/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/opt/anaconda3/envs/regular/include']
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/opt/anaconda3/envs/regular/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/opt/anaconda3/envs/regular/include']
lapack_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/opt/anaconda3/envs/regular/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/opt/anaconda3/envs/regular/include']


## Check the system path

In [1]:
import sys
sys.path